In [ ]:
# ===============================================================
# MODEL TRAINING PIPELINE - POLICY RENEWAL PREDICTION
# ===============================================================

# 🔹 STEP 1: Import libraries
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, roc_curve

import lightgbm as lgb
import shap

# ===============================================================
# STEP 2: Load dataset
# ===============================================================
df = pd.read_csv("../data/renewal_synthetic_dataset.csv")
print("✅ Dataset Loaded - Shape:", df.shape)
df.head()

# ===============================================================
# STEP 3: Define features and target
# ===============================================================
target = "renewed"
X = df.drop(columns=[target, "renewal_prob_synthetic", "customer_id", "last_interaction_date"])
y = df[target]


# ===============================================================
# STEP 4: Split dataset
# ===============================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")


# ===============================================================
# STEP 5: Preprocessing
# ===============================================================
from utils.data_preprocessing import build_preprocessor

preprocessor = build_preprocessor(X_train)
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

print("✅ Preprocessing complete.")


# ===============================================================
# STEP 6: Train LightGBM model
# ===============================================================
model = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=7,
    random_state=42
)
model.fit(X_train_proc, y_train)


# ===============================================================
# STEP 7: Evaluate model
# ===============================================================
y_pred = model.predict(X_test_proc)
y_prob = model.predict_proba(X_test_proc)[:, 1]

metrics = {
    "Accuracy": accuracy_score(y_test, y_pred),
    "Precision": precision_score(y_test, y_pred),
    "Recall": recall_score(y_test, y_pred),
    "F1": f1_score(y_test, y_pred),
    "ROC_AUC": roc_auc_score(y_test, y_prob)
}

print("✅ Model Evaluation Metrics:")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.plot(fpr, tpr, label=f"LightGBM (AUC={metrics['ROC_AUC']:.3f})")
plt.plot([0,1], [0,1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend()
plt.show()


# ===============================================================
# STEP 8: Explain with SHAP
# ===============================================================
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_proc)

plt.title("SHAP Summary Plot")
shap.summary_plot(shap_values, X_test, plot_type="bar")


# ===============================================================
# STEP 9: Save models & preprocessing pipeline
# ===============================================================
joblib.dump(model, "../models/lgb_renewal_model.joblib")
joblib.dump(preprocessor, "../models/preprocessing_objects.joblib")
print("💾 Model and preprocessing objects saved successfully!")


# ===============================================================
# STEP 10: Save sample predictions
# ===============================================================
pred_df = X_test.copy()
pred_df["actual"] = y_test
pred_df["predicted_prob"] = y_prob
pred_df["predicted_label"] = y_pred
pred_df.to_csv("../data/test_predictions_sample.csv", index=False)

print("✅ Predictions saved to '../data/test_predictions_sample.csv'")
